In [1]:
import pandas as pd
from datetime import datetime
import dateutil.relativedelta
# import pypyodbc as odbc

In [127]:
pd.options.mode.copy_on_write = True

# Test database

In [2]:
# drive_name = 'SQL SERVER'
# server_name = 'DESKTOP-29355SM\SQLEXPRESS'
# database_name = 'Bank'

# connection_string = f"""
#     DRIVER={{{drive_name}}};
#     SERVER={server_name};
#     DATABASE={database_name};
#     Trust_Connection=yes;
# """

# conn = odbc.connect(connection_string)

In [3]:
# cursor = conn.cursor()

In [4]:
# cursor.execute("""
# Create table Persons (
#     PersonID int,
#     LastName varchar(255),
#     FirstName varchar(255),
#     Address varchar(255),
#     City varchar(255)
# )
# """)

In [5]:
# df = pd.read_sql('Select * from account', conn)

In [6]:
# conn.commit()

# Function

In [7]:
def extract_date_review(text_date, crawl_date): # Biến đổi khoảng thời gian (1 tháng trước, 2 ngày trước) thằnh ngày cụ thể và trả về datetime
    sub_text = text_date.split(' ')
    date_cmt = ''

    if 'năm' in text_date:
        for t in sub_text:
            if t.isnumeric():
                date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(years=int(t))
    elif 'tháng' in text_date:
        for t in sub_text:
            if t.isnumeric():
                date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(months=int(t))
    elif 'tuần' in text_date:
        for t in sub_text:
            if t.isnumeric():
                date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(weeks==int(t))
    elif 'ngày' in text_date:
        for t in sub_text:
            if t.isnumeric():
                date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date() - dateutil.relativedelta.relativedelta(days=int(t))
    else:
        date_cmt = datetime.strptime(crawl_date, '%Y-%m-%d').date()

    return date_cmt

In [8]:
def create_id(last_id): # Tạo ID mới và trả về string
    last_id_list = []
    newest_id = ''

    for s in list(last_id):
        if s.isnumeric():
            last_id_list.append(s)
        else:
            newest_id += s
    
    last_id = str(int(''.join(last_id_list)) + 1)
    if len(last_id_list) != len(last_id):
        num_0 = len(last_id_list) - len(last_id)
        newest_id += '0'*num_0 + last_id
    else:
        newest_id += last_id
    
    return newest_id

In [84]:
def add_data(file_table_name, df, id_format=None): # Thêm df vào file save(df không chứa cột ID, thứ tự cột giống với file save và nếu file save trống thì cần đưa ID format (S000, PL00,...))
    table = pd.read_csv(r'..\Data\Preprocessed_data\%s'%(file_table_name))

    newest_id = ''
    if id_format != None:
        newest_id = create_id(id_format)
    else:
        newest_id = create_id(table.iloc[-1][0])

    list_id = []
    for i in range(len(df)):
        list_id.append(newest_id)
        newest_id = create_id(newest_id)
    df.insert(loc=0, column='ID', value=list_id)

    df.columns = table.columns
    table = pd.concat([table, df], ignore_index=True)
    table.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    table.to_csv(r'..\Data\Preprocessed_data\%s'%(file_table_name), index=False)

In [120]:
def add_shop(new_shop_df): # Thêm shop mới vào file Shop.csv (new_shop_df chỉ bao gồm 'Shop name', 'Shop rating' và đặt đúng thứ tự)
    shop_df = pd.read_csv(r'..\Data\Preprocessed_data\Shop.csv')
    check_shop_list = new_shop_df.drop_duplicates(subset=new_shop_df.columns[0], keep='last', ignore_index=True)
    drop_index = []

    for index_shop in range(len(check_shop_list)):
        if check_shop_list.iloc[index_shop][0] in shop_df['ShopName'].values:
            drop_index.append(index_shop)

    check_shop_list.drop(drop_index, inplace=True)

    if len(shop_df) == 0:
        add_data('Shop.csv', check_shop_list, id_format='S000')
    else:
        add_data('Shop.csv', check_shop_list)

# Main

In [105]:
platform_df = pd.read_csv(r'..\Data\Preprocessed_data\Platform.csv')
platform_df

,PlatformID,PlatformName
0,PL01,Tiki
1,PL02,Lazada
2,PL03,Shopee
3,PL04,Youtube
4,PL05,Tinhte


In [106]:
product_df = pd.read_csv(r'..\Data\Preprocessed_data\Product.csv')
product_df

,ProductID,ProductName,Brand
0,P001,iPhone 15 Pro Max,iPhone
1,P002,Galaxy S23 Ultra,Samsung


# Tiki

#### Product info

In [170]:
for p in product_df.to_dict(orient='records'):
    product_name = p['ProductName'].lower()

    tiki_product_info = pd.read_csv(r'..\Data\Tiki\%s_info.csv'%('_'.join(product_name.split(' '))))

    # Thêm cột ID của sp
    tiki_product_info['ProductID'] = p['ProductID']

    # Thêm cột ID của platform
    tiki_product_info['PlatformID'] = 'PL01'

    # Lấy những cột cần thiết
    tiki_product_info = tiki_product_info[['ProductID', 'PlatformID', 'Name', 'Quantity', 'Reviews count', 'Star rating', 
                                        'Price', 'Crawl date', 'Shop name', 'Shop rating']]

    # Loại bỏ dấu "." trong cột "Price" và đổi định dạng thành int
    tiki_product_info['Price'] = tiki_product_info['Price'].str.replace('.', '', regex=True).astype(int)

    # Lọc lấy data của ngày crawl mới nhất nếu sp không có sự thay đổi giá
    tiki_product_info.drop_duplicates(subset=['Name', 'Price'], keep='last', inplace=True, ignore_index=True)

    # Thêm shop mới vào file Shop.csv nếu có
    shop_list = tiki_product_info[['Shop name', 'Shop rating']].copy()
    add_shop(shop_list)

    # Thêm ID của shop
    shop_df = pd.read_csv(r'..\Data\Preprocessed_data\Shop.csv')
    for shop_name in shop_df['ShopName']:
        tiki_product_info.loc[tiki_product_info['Shop name'] == shop_name, 'ShopID'] = shop_df[shop_df['ShopName']==shop_name]['ShopID'].values[0]

    add_df = tiki_product_info[['ProductID', 'PlatformID', 'ShopID', 'Price', 'Quantity', 'Reviews count', 'Star rating', 'Crawl date']]

    # Lưu vào file save Market.csv
    market_df = pd.read_csv(r'..\Data\Preprocessed_data\Market.csv')
    if len(market_df) == 0:
        add_data('Market.csv', add_df, 'MK000')
    else:
        add_data('Market.csv', add_df)

#### Reviews

In [21]:
tiki_reviews = pd.read_csv(r'..\Data\Tiki\%s_reviews.csv'%('_'.join(product_name.split(' '))))

In [22]:
tiki_reviews

,Reviewer name,Content,Rating,Rating date,Crawl date,Shop name
0,muasach,"22/9 đúng 00h lên topzone, TGDĐ, FPT Shop, đặt...",Cực kì hài lòng,Đánh giá vào 3 tháng trước,2024-01-11,Tiki Trading
1,Vũ thị hường,Đam mê táo k thể bỏ qua Iphone 15promax vì: \r...,Cực kì hài lòng,Đánh giá vào 3 tháng trước,2024-01-11,Tiki Trading
2,BẢO NGÔ,[Cuộc gọi lúc giữa đêm]…\r\n\r\nKhông phải bon...,Cực kì hài lòng,Đánh giá vào 3 tháng trước,2024-01-11,Tiki Trading
3,NGÔ QUỐC KHÁNH,Tiki giao lúc 1h sáng. Áp mã 500K và giảm 2tr ...,Cực kì hài lòng,Đánh giá vào 3 tháng trước,2024-01-11,Tiki Trading
4,Hung Trieu,Rất tin tưởng Tiki khi đặt mua điện thoại tại ...,Cực kì hài lòng,Đánh giá vào 2 tháng trước,2024-01-11,Tiki Trading
...,...,...,...,...,...,...
294,vu thang,Thấy bảo nhận hàng phải có mã otp mới nhận đc ...,Rất không hài lòng,Đánh giá vào 2 tháng trước,2024-01-27,Tiki Trading
295,Việt Dũng Đoàn,Hàng ko cho đổi trả khi đổi ý,Rất không hài lòng,Đánh giá vào 3 tháng trước,2024-01-27,Tiki Trading
296,Thu Nga,Chưa thấy hoàn astra cho mình,Không hài lòng,Đánh giá vào 3 tháng trước,2024-01-27,Tiki Trading
297,Nguyễn Quang,NaN,Hài lòng,Đánh giá vào 2 tháng trước,2024-01-27,Tiki Trading


In [ ]:
# Biến đổi cột 'Rating date' thành ngày cụ thể thông qua 'Crawl date'
actual_date_list = []

for num in range(len(tiki_reviews)):
    text_date = tiki_reviews.loc[num, ['Rating date', 'Crawl date']][0]
    crawl_date = tiki_reviews.loc[num, ['Rating date', 'Crawl date']][1]

    actual_date = extract_date_review(text_date, crawl_date)
    actual_date_list.append(actual_date)

tiki_reviews['Rating date'] = actual_date_list
tiki_reviews.drop(columns=['Crawl date'], inplace=True)

In [ ]:
# Biến đổi cột "Rating" thành thang đo likert
rating_likert = {'Rất không hài lòng': 1, 'Không hài lòng': 2, 'Bình thường': 3, 'Hài lòng': 4, 'Cực kì hài lòng': 5}

for num in range(len(tiki_reviews)):
    rating = tiki_reviews.loc[num, 'Rating']
    for k, v in rating_likert.items():
        if rating == k:
            tiki_reviews.loc[num, 'Rating'] = v

In [ ]:
# Loại bỏ những dòng trùng "Reviewer name" và "Content"
tiki_reviews.drop_duplicates(subset=['Reviewer name', 'Content'], keep='last', inplace=True, ignore_index=True)

In [ ]:
tiki_reviews

,Reviewer name,Content,Rating,Rating date,Shop name
0,Lý Nguyen,😂. Haha,5,2023-12-23,Tiki Trading
1,vu thang,Thấy bảo nhận hàng phải có mã otp mới nhận đc ...,1,2023-12-11,Tiki Trading
2,muasach,"22/9 đúng 00h lên topzone, TGDĐ, FPT Shop, đặt...",5,2023-09-27,Tiki Trading
3,Vũ thị hường,Đam mê táo k thể bỏ qua Iphone 15promax vì: \r...,5,2023-09-27,Tiki Trading
4,BẢO NGÔ,[Cuộc gọi lúc giữa đêm]…\r\n\r\nKhông phải bon...,5,2023-09-27,Tiki Trading
...,...,...,...,...,...
96,vu thang,Thấy bảo nhận hàng phải có mã otp mới nhận đc ...,1,2023-11-27,Tiki Trading
97,Việt Dũng Đoàn,Hàng ko cho đổi trả khi đổi ý,1,2023-10-27,Tiki Trading
98,Thu Nga,Chưa thấy hoàn astra cho mình,2,2023-10-27,Tiki Trading
99,Nguyễn Quang,NaN,4,2023-11-27,Tiki Trading


# Lazada

# Websites

#### Tinhte

In [ ]:
tinhte_content = pd.read_csv(r'..\Data\TinhTe\TinhTe_Iphone15pm_InfoPosts2.csv')

# Youtube